In [2]:
import numpy as np
import cv2

## Zadanie 1 (popraw parametry)

In [4]:
def median(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = np.ones((kernel_1, kernel_1), np.uint8)
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)


    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    size = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start)).shape
    f.close()
    BUF = np.zeros((size[0],size[1],60),np.uint8)

    for ii in range(roi_start-60,roi_start):
        I_VIS = cv2.imread("%s/input/in%06d.jpg" % (name, ii))
        BUF[:, :, iN] = cv2.cvtColor(I_VIS, cv2.COLOR_BGR2GRAY)
        iN = iN + 1
        if (iN == 60):
            iN = 0

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.median(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=2)
        thresh = cv2.dilate(thresh, kernel1, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Mediana:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

def average(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = np.ones((kernel_1, kernel_1), np.uint8)
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    iN = 0

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    size = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start)).shape
    f.close()
    BUF = np.zeros((size[0],size[1],60),np.uint8)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        BUF[:,:,iN] = I
        iN = iN + 1
        if(iN == 60):
            iN = 0

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_back = np.average(BUF, axis=2).astype(np.uint8)
        I_diff = cv2.absdiff(I, I_back)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Średnia:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [8]:
median("highway", 5, 36, 2, 4, 9)
average("highway", 5, 36, 3, 7, 11)

Mediana:
P =  0.7474609148934094
R =  0.9854144700737871
F1 =  0.850100137309643
Średnia:
P =  0.25873566008256565
R =  0.9191448524119394
F1 =  0.40380250386629085


In [7]:
median("pedestrian", 5, 30, 3, 4, 9)
average("pedestrian", 5, 30, 3, 4, 9)

Mediana:
P =  0.6244367237200849
R =  0.9873070043482491
F1 =  0.7650232978911568
Średnia:
P =  0.027710043728346213
R =  0.9142698908073075
F1 =  0.05378980533437659


## Zadanie 2

In [9]:
def median_nonbuf(name, speed, threshold_value, kernel_1, kernel_2, blur):
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    mask = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        mask = np.where(mask < I, mask + 1, np.where(mask > I, mask - 1, mask))

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        I_diff = cv2.absdiff(I, mask)

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Mediana:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

def average_nonbuf(name, speed, threshold_value, kernel_1, kernel_2, blur, alpha):
    kernel1 = np.ones((kernel_1, kernel_1), np.uint8)
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    BUF = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    BUF = cv2.cvtColor(BUF, cv2.COLOR_BGR2GRAY)
    BUF = BUF.astype(np.float64)
    f.close()


    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        BUF = alpha*I+(1-alpha)*BUF
        I_diff = cv2.absdiff(I, BUF.astype(np.uint8))

        (T,thresh) = cv2.threshold(I_diff,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Średnia:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [10]:
median_nonbuf("highway", 5, 36, 5, 3, 11)

Mediana:
P =  0.9932660306806754
R =  0.49299863309491393
F1 =  0.6589388920560656


In [6]:
average_nonbuf("highway", 5, 36, 5, 3, 11,0.05)

Średnia:
P =  0.9635581635715
R =  0.3453572567768433
F1 =  0.5084695297232041


## Zadanie 4

In [6]:
def mog(name, speed, threshold_value, kernel_1, kernel_2, blur):
    mog = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=36, detectShadows=True)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    temp_1 = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    temp_1 = cv2.cvtColor(temp_1, cv2.COLOR_BGR2GRAY)
    temp_2 = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start+1))
    temp_2 = cv2.cvtColor(temp_2, cv2.COLOR_BGR2GRAY)
    mask = cv2.absdiff(temp_1, temp_2)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        mask = mog.apply(I, mask, -1)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        (T,thresh) = cv2.threshold(mask,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("Mog:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [19]:
mog("highway", 5, 16, 5, 4, 11)

Mog:
P =  0.8827484775364817
R =  0.5296568764885891
F1 =  0.6620674440302795


In [18]:
mog("pedestrian", 5, 25, 3, 4, 7)

Mog:
P =  0.5491476485844503
R =  0.9793065253672811
F1 =  0.7036964337091521


In [10]:
mog("office", 5, 6, 3, 7, 11)

Mog:
P =  0.6087217721945585
R =  0.14425010604923347
F1 =  0.23323096846151398


## Zadanie 5

In [3]:
def knn(name, speed, threshold_value, kernel_1, kernel_2, blur):
    knn = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=36, detectShadows=True)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(kernel_1,kernel_1))
    kernel2 = np.ones((kernel_2, kernel_2), np.uint8)

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    f = open("{}/temporalROI.txt".format(name), "r")
    line = f.readline()
    roi_start, roi_end = line.split()
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    f.close()
    temp_1 = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start))
    temp_1 = cv2.cvtColor(temp_1, cv2.COLOR_BGR2GRAY)
    temp_2 = cv2.imread("%s/input/in%06d.jpg" % (name, roi_start+1))
    temp_2 = cv2.cvtColor(temp_2, cv2.COLOR_BGR2GRAY)
    mask = cv2.absdiff(temp_1, temp_2)

    for i in range(roi_start, roi_end, speed):
        I = cv2.imread("%s/input/in%06d.jpg" % (name, i))
        I_vis = I
        I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)

        mask = knn.apply(I, mask, -1)

        I_ref = cv2.imread("%s/groundtruth/gt%06d.png" % (name, i))
        I_ref = cv2.cvtColor(I_ref,cv2.COLOR_BGR2GRAY)
        cv2.imshow("I_ref",I_ref)

        (T,thresh) = cv2.threshold(mask,threshold_value,255,cv2.THRESH_BINARY)
        thresh = cv2.erode(thresh, kernel1, iterations=1)
        thresh = cv2.dilate(thresh, kernel2, iterations=1)
        thresh = cv2.medianBlur(thresh, blur)

                #Index
        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh)
        labels = np.uint8(labels / retval * 255)

        #Rectangle
        if(stats.shape[0] > 1):
            tab = stats[1: ,4]
            pi = np.argmax(tab)
            pi = pi + 1
            cv2.rectangle(I_vis,(stats[pi,0],stats[pi,1]),(stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]),(255,0,0),2)
            cv2.putText(I_vis, str(stats[pi,4]), (stats[pi,0],stats[pi,1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)
            cv2.putText(I_vis, str(pi), (int(centroids[pi, 0]),int(centroids[pi, 1])),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))

        TP_M = np.logical_and((thresh == 255),(I_ref == 255))
        TP_S = np.sum(TP_M)
        TP = TP + TP_S
        
        FP_M = np.logical_and((thresh == 255),(I_ref == 0))
        FP_S = np.sum(FP_M)
        FP = FP + FP_S

        FN_M = np.logical_and((thresh == 0),(I_ref == 255))
        FN_S = np.sum(FN_M)
        FN = FN + FN_S

        cv2.imshow("I",I_vis)  
        cv2.imshow("Raw",thresh)
        cv2.waitKey(15)
    cv2.destroyAllWindows()

    P = TP/(TP+FP)
    R = TP/(TP+FN)
    F1 = 2*P*R/(P+R)
    print("KNN:")
    print("P = ", P)
    print("R = ", R)
    print("F1 = ", F1)

In [24]:
knn("highway", 5, 16, 5, 4, 11)

KNN:
P =  0.7214128718961164
R =  0.9250573833557474
F1 =  0.8106411901055458


In [25]:
knn("pedestrian", 5, 25, 3, 4, 7)

KNN:
P =  0.23950013368768044
R =  0.9988699042786472
F1 =  0.38636185998847855


In [4]:
knn("office", 5, 6, 3, 4, 11)

KNN:
P =  0.4332967135050426
R =  0.6208408590190461
F1 =  0.5103855717398839
